In [1]:
!pip install transformers[torch]
# !pip install torch
!pip install datasets
!pip install langchain scikit-learn
!pip install gdown

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 8.1 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [2]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [3]:

import gdown

url = 'https://drive.google.com/uc?id=1Ia_4o4JJDoyIkPcbMC_eZbWE_1DpuerL'
output = 'data.zip'
gdown.download(url, output, quiet=False)


Downloading...
From: https://drive.google.com/uc?id=1Ia_4o4JJDoyIkPcbMC_eZbWE_1DpuerL
To: /content/data.zip
100%|██████████| 12.2M/12.2M [00:00<00:00, 22.1MB/s]


'data.zip'

In [4]:
import zipfile
import os

with zipfile.ZipFile(output, 'r') as zip_ref:
    zip_ref.extractall('/content')



In [5]:
import pandas as pd

# Adjust the path below if the speech.csv is inside a folder from the zip
csv_file = '/content/data/updated_file_pre_2006.csv'
data = pd.read_csv(csv_file)

len(data)
data = data[:180]
len(data)

180

In [6]:
from google.colab import drive
from transformers import AutoTokenizer, AutoModelForMaskedLM, BertModel
import torch
import random
from nltk.corpus import stopwords
from nltk import word_tokenize, pos_tag

# Mount Google Drive
drive.mount('/content/drive')

# Path to the fine-tuned model on Google Drive
model_path = "/content/drive/My Drive/finbert_finetuned"

# Load the fine-tuned model and tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = BertModel.from_pretrained(model_path)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)  # Ensure the model is on the correct device


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Some weights of BertModel were not initialized from the model checkpoint at /content/drive/My Drive/finbert_finetuned and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False

In [7]:
import pandas as pd

# Load dataset
df = data
df = df[['content', 'Classification']]

# Transform classification labels
def transform_classification_label(label):
    return 1 if label == 1.0 else 0

df['classification'] = df['Classification'].apply(transform_classification_label)

<ipython-input-7-5f0e532300ea>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['classification'] = df['Classification'].apply(transform_classification_label)


In [8]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Token count splitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=512,
    chunk_overlap=50,
    separators=["\n\n", "\n", ".", "!", "?", ",", " "]
)

def split_text(text):
    chunks = text_splitter.split_text(text)
    return chunks

# Apply text splitting to the dataframe
df['chunks'] = df['content'].apply(split_text)


<ipython-input-8-983f467ded78>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['chunks'] = df['content'].apply(split_text)


In [9]:
import torch
from tqdm import tqdm

def get_cls_embeddings(texts, tokenizer, model, device):
    # Tokenize the input texts and prepare inputs
    inputs = tokenizer(texts, return_tensors='pt', padding=True, truncation=True, max_length=512)
    # Move all inputs to the specified device (GPU or CPU)
    inputs = {key: value.to(device) for key, value in inputs.items()}

    # Perform model inference without calculating gradients
    with torch.no_grad():
        outputs = model(**inputs)
        # Extract the CLS token's embeddings from the last hidden state
        cls_embeddings = outputs.last_hidden_state[:, 0, :]  # [batch_size, hidden_size]
    # Move embeddings back to CPU for further processing/storage (if necessary)
    return cls_embeddings.cpu().numpy()

# Load model and tokenizer onto the specified device
model.to(device)  # Make sure your model is also on the same device before the loop

# Prepare lists to hold the embeddings and labels
cls_embeddings_list = []
labels_list = []

# Iterate over the DataFrame to process each row
for _, row in tqdm(df.iterrows(), total=len(df)):
    chunks = row['chunks']  # Assume 'chunks' contains a list of text segments
    label = row['classification']

    # Process each list of text chunks
    embeddings = get_cls_embeddings(chunks, tokenizer, model, device)
    cls_embeddings_list.extend(embeddings)
    labels_list.extend([label] * len(embeddings))
    torch.cuda.empty_cache()  # Clear unused memory



100%|██████████| 180/180 [02:58<00:00,  1.01it/s]


In [10]:
import numpy as np
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import Dataset, DataLoader

# Convert lists to numpy arrays
X = np.array(cls_embeddings_list)
y = np.array(labels_list)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define a custom dataset
class TextDataset(Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return torch.tensor(self.data[idx], dtype=torch.float32), torch.tensor(self.labels[idx], dtype=torch.long)

# Create DataLoader objects
train_dataset = TextDataset(X_train, y_train)
test_dataset = TextDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


In [11]:
import torch.nn as nn

class LSTMClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers):
        super(LSTMClassifier, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = x.unsqueeze(1)  # Add batch dimension for LSTM
        lstm_out, _ = self.lstm(x)
        out = lstm_out[:, -1, :]  # Get the last output of LSTM
        out = self.fc(out)
        return out

# Instantiate the model
input_dim = X.shape[1]
hidden_dim = 128
output_dim = 2  # Positive or Negative
num_layers = 2

lstm_model = LSTMClassifier(input_dim, hidden_dim, output_dim, num_layers).to(device)


In [12]:
import torch.optim as optim

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(lstm_model.parameters(), lr=0.001)

# Training function
def train_model(model, train_loader, criterion, optimizer, num_epochs):
    model.train()
    for epoch in range(num_epochs):
        total_loss = 0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()  # Zero gradients

            outputs = model(inputs)  # Forward pass

            loss = criterion(outputs, labels)  # Compute loss

            loss.backward()  # Backward pass
            optimizer.step()  # Optimize

            total_loss += loss.item()

        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss / len(train_loader):.4f}")

# Train the model
train_model(lstm_model, train_loader, criterion, optimizer, num_epochs=10)


Epoch [1/10], Loss: 0.5005
Epoch [2/10], Loss: 0.2626
Epoch [3/10], Loss: 0.1622
Epoch [4/10], Loss: 0.1204
Epoch [5/10], Loss: 0.0981
Epoch [6/10], Loss: 0.0833
Epoch [7/10], Loss: 0.0719
Epoch [8/10], Loss: 0.0659
Epoch [9/10], Loss: 0.0572
Epoch [10/10], Loss: 0.0534


In [13]:
from sklearn.metrics import classification_report, accuracy_score

# Evaluation function
def evaluate_model(model, test_loader, criterion):
    model.eval()
    all_preds = []
    all_labels = []
    total_loss = 0

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)

            loss = criterion(outputs, labels)
            total_loss += loss.item()

            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    accuracy = accuracy_score(all_labels, all_preds)
    report = classification_report(all_labels, all_preds)

    print(f"Loss: {total_loss / len(test_loader):.4f}")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Classification Report:\n{report}")

# Evaluate the LSTM model
evaluate_model(lstm_model, test_loader, criterion)


Loss: 0.1714
Accuracy: 0.9454
Classification Report:
              precision    recall  f1-score   support

           0       0.92      0.99      0.95     15187
           1       0.98      0.90      0.94     14031

    accuracy                           0.95     29218
   macro avg       0.95      0.94      0.95     29218
weighted avg       0.95      0.95      0.95     29218



In [14]:
path_to_save_model = '/content/drive/My Drive/nlp_lstm_finetuned.pth'
torch.save(lstm_model.state_dict(), path_to_save_model)
print(f"Model saved to {path_to_save_model}")


Model saved to /content/drive/My Drive/nlp_lstm_finetuned.pth
